In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import estimator
import pandas as pd
import numpy as np


from keras import backend as K

#Clear TF memory
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

wine_data = load_wine()

C:\Users\StrikeWade\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
wine_data

{'DESCR': 'Wine Data Database\n====================\n\nNotes\n-----\nData Set Characteristics:\n    :Number of Instances: 178 (50 in each of three classes)\n    :Number of Attributes: 13 numeric, predictive attributes and the class\n    :Attribute Information:\n \t\t- 1) Alcohol\n \t\t- 2) Malic acid\n \t\t- 3) Ash\n\t\t- 4) Alcalinity of ash  \n \t\t- 5) Magnesium\n\t\t- 6) Total phenols\n \t\t- 7) Flavanoids\n \t\t- 8) Nonflavanoid phenols\n \t\t- 9) Proanthocyanins\n\t\t- 10)Color intensity\n \t\t- 11)Hue\n \t\t- 12)OD280/OD315 of diluted wines\n \t\t- 13)Proline\n        \t- class:\n                - class_0\n                - class_1\n                - class_2\n\t\t\n    :Summary Statistics:\n    \n    ============================= ==== ===== ======= =====\n                                   Min   Max   Mean     SD\n    ============================= ==== ===== ======= =====\n    Alcohol:                      11.0  14.8    13.0   0.8\n    Malic Acid:                   0.74  5.80   

In [3]:
wine_data.keys()

dict_keys(['DESCR', 'target_names', 'target', 'feature_names', 'data'])

In [4]:
print(wine_data['DESCR'])

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

In [5]:
wine_data['target_names']

array(['class_0', 'class_1', 'class_2'], dtype='<U7')

In [6]:
feature_data = wine_data['data']
labels = wine_data['target']

In [7]:
#Split data
x_train, x_test, y_train, y_test = train_test_split(feature_data, labels, test_size=0.3, random_state=101)

In [8]:
#Scale data
scaler = MinMaxScaler()

In [9]:
scaled_x_train = scaler.fit_transform(x_train)
scaled_x_test = scaler.transform(x_test) # dont want to assume that we are going to know future test data 

# Estimator API

In [10]:
#Set Feature columns
feat_cols = [tf.feature_column.numeric_column('x', shape=[13])]

In [11]:
#Model
deep_model = estimator.DNNClassifier(hidden_units=[13,13,13], 
                                     feature_columns=feat_cols, 
                                     n_classes=3, 
                                     optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E0D0EAA908>, '_task_id': 0, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_master': '', '_save_summary_steps': 100, '_model_dir': 'C:\\Users\\STRIKE~1\\AppData\\Local\\Temp\\tmpybb0h39j', '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_service': None, '_task_type': 'worker', '_num_worker_replicas': 1, '_is_chief': True, '_session_config': None}


In [12]:
#Input function
input_function = estimator.inputs.numpy_input_fn(x={'x': scaled_x_train}, 
                                                 y=y_train, 
                                                 shuffle=True, 
                                                 batch_size=10, 
                                                 num_epochs=10)

#Input evaluation function
input_eval_function = estimator.inputs.numpy_input_fn(x={'x': scaled_x_test},  
                                                      shuffle=False)

In [13]:
#Train the model
deep_model.train(input_fn=input_function, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\STRIKE~1\AppData\Local\Temp\tmpybb0h39j\model.ckpt.
INFO:tensorflow:loss = 10.766004, step = 1
INFO:tensorflow:global_step/sec: 242.683
INFO:tensorflow:loss = 3.638844, step = 101 (0.418 sec)
INFO:tensorflow:Saving checkpoints for 124 into C:\Users\STRIKE~1\AppData\Local\Temp\tmpybb0h39j\model.ckpt.
INFO:tensorflow:Loss for final step: 0.8468193.


In [14]:
#Cast the evaluation predicts into a list
y_pred = list(deep_model.predict(input_fn=input_eval_function))

INFO:tensorflow:Restoring parameters from C:\Users\STRIKE~1\AppData\Local\Temp\tmpybb0h39j\model.ckpt-124


In [15]:
y_pred

[{'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logits': array([ 1.5286713 ,  0.94423306, -3.3791203 ], dtype=float32),
  'probabilities': array([0.63905615, 0.356222  , 0.00472185], dtype=float32)},
 {'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logits': array([ 1.9624023,  1.4077007, -4.25992  ], dtype=float32),
  'probabilities': array([0.63442594, 0.364315  , 0.0012591 ], dtype=float32)},
 {'class_ids': array([2], dtype=int64),
  'classes': array([b'2'], dtype=object),
  'logits': array([-3.8484437 , -0.39111948,  4.0189137 ], dtype=float32),
  'probabilities': array([3.7830224e-04, 1.2004267e-02, 9.8761743e-01], dtype=float32)},
 {'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logits': array([ 3.7148364,  0.4186728, -5.33304  ], dtype=float32),
  'probabilities': array([9.6418756e-01, 3.5698984e-02, 1.1342760e-04], dtype=float32)},
 {'class_ids': array([2], dtype=int64),


In [16]:
predictions = [p['class_ids'][0] for p in y_pred]
(predictions,y_test)

([0,
  0,
  2,
  0,
  2,
  1,
  2,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  2,
  1,
  1,
  1,
  2,
  2,
  2,
  0,
  0,
  1,
  1,
  2,
  1,
  2,
  2,
  2,
  0,
  1,
  0,
  0,
  1,
  1,
  2,
  1,
  0,
  0,
  1,
  2,
  1,
  1,
  2,
  2,
  1,
  0,
  1,
  1,
  0],
 array([0, 0, 2, 0, 2, 1, 2, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 2, 1, 1, 1, 2,
        2, 2, 0, 0, 1, 1, 2, 1, 2, 1, 1, 0, 0, 0, 0, 1, 1, 2, 1, 0, 0, 1,
        2, 1, 1, 2, 2, 1, 0, 1, 1, 0]))

In [17]:
print(classification_report(y_pred=predictions, y_true=y_test))

             precision    recall  f1-score   support

          0       1.00      0.89      0.94        19
          1       0.91      0.91      0.91        22
          2       0.87      1.00      0.93        13

avg / total       0.93      0.93      0.93        54



In [18]:
print(confusion_matrix(y_pred=predictions, y_true=y_test))

[[17  2  0]
 [ 0 20  2]
 [ 0  0 13]]


# Keras API

In [19]:
from tensorflow.contrib.keras import models, layers, losses, optimizers, metrics, activations

In [20]:
#Create DNN keras model
dnn_keras_model = models.Sequential()

In [21]:
#Add 3 Dense layers
dnn_keras_model.add(layers.Dense(units=13, input_dim=13, activation='relu'))
dnn_keras_model.add(layers.Dense(units=13, activation='relu'))
dnn_keras_model.add(layers.Dense(units=13, activation='relu'))

In [22]:
#Add the last layer with 3 output
dnn_keras_model.add(layers.Dense(units=3, activation='softmax'))

In [23]:
#Compile all the layers
# loss: depend on format of the data - 
#    Not Onehot->sparse_categorical_crossentropy, 
#        Onehot->categorical_crossentropy
dnn_keras_model.compile(optimizer='adam', 
                        loss='sparse_categorical_crossentropy', 
                        metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [24]:
#Train the model
dnn_keras_model.fit(scaled_x_train, y_train, epochs=100)

Epoch 1/100
124/124 [==============================]124/124 [==============================] - 0s 3ms/step - loss: 1.1088 - acc: 0.2984

Epoch 2/100
124/124 [==============================]124/124 [==============================] - 0s 169us/step - loss: 1.0872 - acc: 0.3226

Epoch 3/100
124/124 [==============================]124/124 [==============================] - 0s 161us/step - loss: 1.0669 - acc: 0.3226

Epoch 4/100
124/124 [==============================]124/124 [==============================] - 0s 153us/step - loss: 1.0479 - acc: 0.3710

Epoch 5/100
124/124 [==============================]124/124 [==============================] - 0s 161us/step - loss: 1.0309 - acc: 0.4355

Epoch 6/100
124/124 [==============================]124/124 [==============================] - 0s 153us/step - loss: 1.0122 - acc: 0.4839

Epoch 7/100
124/124 [==============================]124/124 [==============================] - 0s 153us/step - loss: 0.9963 - acc: 0.5403

Epoch 8/100
124/124 [========

In [25]:
predictions0 = dnn_keras_model.predict(scaled_x_test)
predictions0

array([[9.10455465e-01, 8.87314603e-02, 8.13062652e-04],
       [9.21810269e-01, 7.79261962e-02, 2.63476773e-04],
       [3.93592811e-04, 1.84685446e-03, 9.97759581e-01],
       [9.99775827e-01, 2.21867056e-04, 2.29738498e-06],
       [3.36274737e-04, 7.27973413e-04, 9.98935759e-01],
       [9.36970487e-03, 9.64403689e-01, 2.62265373e-02],
       [1.22324424e-03, 8.71241558e-03, 9.90064323e-01],
       [9.99176323e-01, 8.11891630e-04, 1.18144899e-05],
       [6.25344366e-02, 9.37127650e-01, 3.37938778e-04],
       [6.81866586e-01, 3.17505658e-01, 6.27660949e-04],
       [9.99781072e-01, 2.12574232e-04, 6.30699151e-06],
       [1.03302300e-01, 8.93347681e-01, 3.35001224e-03],
       [7.50116348e-01, 2.44618952e-01, 5.26471250e-03],
       [9.65068400e-01, 3.40753533e-02, 8.56235623e-04],
       [9.98751760e-01, 1.23092718e-03, 1.72047148e-05],
       [9.99354661e-01, 6.35098142e-04, 1.02019640e-05],
       [1.86842531e-02, 9.77781475e-01, 3.53429560e-03],
       [6.65343716e-04, 1.66294

In [26]:
predictions1 = dnn_keras_model.predict_classes(scaled_x_test)
predictions1

array([0, 0, 2, 0, 2, 1, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 2, 1, 1, 1, 2,
       2, 2, 0, 0, 1, 1, 2, 1, 2, 2, 2, 0, 1, 0, 0, 1, 1, 2, 1, 0, 0, 0,
       2, 1, 1, 2, 2, 1, 0, 1, 1, 0], dtype=int64)

In [27]:
print(classification_report(y_pred=predictions1, y_true=y_test))
print(confusion_matrix(y_pred=predictions1, y_true=y_test))

             precision    recall  f1-score   support

          0       0.90      0.95      0.92        19
          1       0.95      0.82      0.88        22
          2       0.87      1.00      0.93        13

avg / total       0.91      0.91      0.91        54

[[18  1  0]
 [ 2 18  2]
 [ 0  0 13]]


# Layers API

In [69]:
tf.reset_default_graph()

In [70]:
from tensorflow.contrib.layers import fully_connected

In [71]:
y_train

array([1, 1, 1, 0, 2, 2, 0, 1, 0, 0, 2, 2, 2, 2, 1, 2, 0, 0, 1, 1, 2, 0,
       1, 2, 0, 1, 2, 0, 0, 1, 1, 1, 0, 1, 1, 2, 0, 2, 2, 1, 2, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 2, 0, 1, 2, 0, 0, 2, 2, 0, 0, 0, 1, 2, 2,
       0, 0, 1, 1, 1, 2, 2, 1, 0, 1, 1, 0, 2, 2, 1, 1, 0, 1, 2, 0, 2, 0,
       1, 1, 0, 1, 1, 2, 1, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 1, 0, 2, 2,
       0, 1, 1, 2, 2, 0, 2, 1, 1, 1, 1, 1, 0, 1])

In [72]:
# convert y_train to onehot -> as_matrix(), format that Layers API needs
onehot_y_train = pd.get_dummies(y_train).as_matrix()
onehot_y_test =pd.get_dummies(y_test).as_matrix()

In [73]:
#Define network Params
num_features_inputs = 13 #13 features inputs
num_hidden1 = 13 #13 neurons
num_hidden2 = 13 #13 neurons
num_outputs = 3 #3 classes

learning_rate = 0.01
epochs = 2

#Placeholder
x = tf.placeholder(dtype=tf.float32, shape=[None, num_features_inputs])
y = tf.placeholder(dtype=tf.float32, shape=[None, num_outputs])

In [74]:
#Activation function
activation_function = tf.nn.relu

hidden1 = fully_connected(x, num_hidden1, activation_fn=activation_function)
hidden2 = fully_connected(hidden1, num_hidden2, activation_fn=activation_function)

outputs = fully_connected(hidden2, num_outputs)

In [75]:
#Loss function
loss = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=outputs)

#Optimizer function
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [76]:
#Init function
init = tf.global_variables_initializer()

In [77]:
#Training phase
with tf.Session() as sess:
    
    sess.run(init)
    
    for epoch in range(epochs):
        
        sess.run(optimizer, feed_dict={x: scaled_x_train, y: onehot_y_train})
        
    predictions = outputs.eval(feed_dict={x: scaled_x_test})
    
    y_preds = tf.argmax(predictions, axis=1)
    
    results = y_preds.eval()

In [78]:
results

array([2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2,
       2, 1, 1, 2, 2, 1, 2, 1, 2, 2], dtype=int64)

In [79]:
print(classification_report(y_pred=results, y_true=y_test))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        19
          1       1.00      0.55      0.71        22
          2       0.31      1.00      0.47        13

avg / total       0.48      0.46      0.40        54



C:\Users\StrikeWade\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
